In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from stog.utils.params import Params
from stog.data.dataset_builder import dataset_from_params, iterator_from_params
from stog.data.vocabulary import Vocabulary
from stog.training.trainer import Trainer
from stog.data.dataset import Batch

[2020-02-09 18:07:48,660 INFO] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [3]:
params = Params.from_file("../model/model_params.yaml")

/home/adylanrff/Documents/Kuliah/TA/amr_parser/stog/utils/params.py:104: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dict_merge.dict_merge(params_dict, yaml.load(f))


In [4]:
data_params = params['data']
dataset = dataset_from_params(data_params)

[2020-02-09 18:07:48,746 INFO] Building train datasets ...
[2020-02-09 18:07:48,747 ERROR] Model name 'data/bert-base-cased/bert-base-cased-vocab.txt' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese). We assumed 'data/bert-base-cased/bert-base-cased-vocab.txt' was a path or url but couldn't find any file associated to this path or url.
0it [00:00, ?it/s][2020-02-09 18:07:48,750 INFO] Reading instances from lines in file at: ../data/raw/amr.txt.features
[2020-02-09 18:07:48,825 INFO] POS tag coverage: 0.3087 (184/596)
40it [00:00, 526.23it/s]
[2020-02-09 18:07:48,826 INFO] Building dev datasets ...
[2020-02-09 18:07:48,828 ERROR] Model name 'data/bert-base-cased/bert-base-cased-vocab.txt' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-

In [36]:
train_data = dataset['train']
dev_data = dataset.get('dev')
test_data = dataset.get('test')

TypeError: 'Batch' object is not subscriptable

In [37]:
vocab_params = params.get('vocab', {})
vocab = Vocabulary.from_instances(instances=train_data, **vocab_params)
vocab.save_to_files("../data/processed/serialization")

dataset = Batch(train_data)
dataset.index_instances(vocab)
print(dataset.as_tensor_dict()['src_tokens']['encoder_tokens'][0].numpy())
print(dataset.as_tensor_dict()['tgt_tokens']['decoder_tokens'][0].numpy())
print(dataset.get_padding_lengths())
# print(dataset.as_tensor_dict())

train_iterator, dev_iterater, test_iterater = iterator_from_params(vocab, data_params['iterator'])

[2020-02-09 19:06:23,823 INFO] Fitting token dictionary from dataset.
100%|██████████| 40/40 [00:00<00:00, 2299.16it/s]
[2020-02-09 19:06:23,850 WARNING] vocabulary serialization directory ../data/processed/serialization is not empty


[19 20 31 74 75  3 76 77 32  3 78 79 80  5 81 82  2  0  0  0  0  0  0  0
  0]
[ 3 65  5  2 66 67  5 68 69 70 71 72 73  4  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]
{'src_tokens': {'num_tokens': 25, 'num_token_characters': 14}, 'src_must_copy_tags': {'num_tokens': 25}, 'tgt_tokens': {'num_tokens': 29, 'num_token_characters': 17}, 'src_pos_tags': {'num_tokens': 25}, 'tgt_pos_tags': {'num_tokens': 29}, 'tgt_copy_indices': {'num_tokens': 29}, 'tgt_copy_mask': {'num_tokens': 29}, 'tgt_copy_map': {'num_tokens': 29}, 'src_copy_indices': {'num_tokens': 29}, 'src_copy_map': {'num_tokens': 27}, 'head_tags': {'num_tokens': 29}, 'head_indices': {'num_tokens': 29}}


In [38]:
train_iterator.